[Reference](https://towardsdatascience.com/scrape-data-from-pdf-files-using-python-fe2dc96b1e68)

In [1]:
!pip install tabula-py
!pip install pandas

     |████████████████████████████████| 11.7MB 248kB/s 


In [2]:
import tabula as tb
import pandas as pd
import re

In [3]:
file = 'state_population.pdf'
data = tb.read_pdf(file, area = (300, 0, 600, 800), pages = '1')

In [4]:
file = 'payroll_sample.pdf'
df= tb.read_pdf(file, pages = '1', area = (0, 0, 1000, 1000), columns = [200, 265, 300, 320], pandas_options={'header': None}, stream=True)[0]

In [5]:
df['border'] = df.apply(lambda x: 1 if re.findall('^[A-Z].*[a-z]$', str(x[0])) else 0, axis = 1)
df['row'] = df['border'].transform('cumsum')

In [6]:
# reshape left section
employee = df[[0, 'row']]
employee = employee[employee[0].notnull()]
employee['index'] = employee.groupby('row').cumcount()+1
employee = employee.pivot(index = ['row'], columns = ['index'], values = 0).reset_index()
employee = employee.rename(columns = {1: 'employee_name', 2: 'net_amount', 3: 'pay_date', 4: 'pay_period'})
employee['net_amount'] = employee.apply(lambda x: x['net_amount'].replace('Net', '').strip(), axis = 1)# reshape right section
payment = df[[1, 2, 3, 4, 'row']]
payment = payment[payment[1].notnull()]
payment = payment[payment['row']!=0]
payment = payment.pivot(index = ['row'], columns = 1, values = [2, 3, 4]).reset_index()
payment.columns = [str(col[0])+col[1] for col in payment.columns.values]
for i in ['Regular', 'OT']:
    payment = payment.rename(columns = {f'2{i}': f'{i}_Rate', f'3{i}': f'{i}_Hours', f'4{i}': f'{i}_Amt'})

In [7]:
df_clean = employee.merge(payment, on = ['row'], how = 'inner')